In [ ]:
import os
import cv2

import pandas as pd
import rasterio as rio

from tqdm import tqdm
from pathlib import Path
from shapely import wkt
from rasterio import features

In [ ]:
dataset_path = Path('../../change_detection_dataset/SN7_buildings/train')
df_path = '../../building_detection_dataset/dataset_1/all_df.csv'

output_path = Path('../../building_detection_dataset/SN7_masks')

### Read csv

In [ ]:
df = pd.read_csv(df_path, index_col=0)

In [ ]:
df

### Generate masks

In [ ]:
os.makedirs(output_path, exist_ok=True)

In [ ]:
AOI_names = list(set(df['AOI_name']))

for AOI_name in tqdm(AOI_names):
    os.makedirs(output_path / AOI_name, exist_ok=True)

    file_names = sorted(list(set(df[df['AOI_name'] == AOI_name]['filename'])))
    
    for file_name in file_names:
        image_path = dataset_path / AOI_name / 'images' / '{}.tif'.format(file_name)

        r = rio.open(image_path).read()

        image = r.transpose((1, 2, 0))[:, :, :-1]

        polygons = list(df[(df['AOI_name'] == AOI_name) &
                        (df['filename'] == file_name)]['geometry'])
        
        mask = features.rasterize(((wkt.loads(polygon), 255) for polygon in polygons),
                                  out_shape=image.shape[:2])

        cv2.imwrite(str(output_path / AOI_name / '{}.png'.format(file_name)), mask)